In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoConfig,AutoModel
import torch


d:\anaconda\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\anaconda\envs\llm\lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
model_name = "../../model/gpt2"
CLLM = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer= AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
llm = AutoModel.from_pretrained(model_name, trust_remote_code=True,output_hidden_states=True,output_attentions=True)

### AutoModelForCausalLM 跟 AutoModel的区别

#### AutoModelForCausalLM 跟 AutoModel的区别
- 多了一个MLPHead,维度为(outputsize,vocal_size)
- 即输出到词表的映射

#### gpt2属于自回归模型
- 对于输入x,得到$y_1$,然后将[x,$y_1$]作为输入,接着得到$y_2$

In [47]:
CLLM

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [48]:
llm

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

### greed_search vs. beam_search

#### greed_search vs. beam_search
- greed_search:贪心策略，对于每次的输入y,总是选择概率最大的那个,弊端：输出缺乏多样性,场景：数学(要求精确，而不是多样)
- beam_search:束搜索，输入的时候进行束(k)展开，每次输出都进行k展开，最后计算得到最大概率的输出

In [ ]:
import torch
import pandas as pd

def predict_next_tokens(model, tokenizer, sentence, num_steps=10, top_k=5):
    model.eval()

    inputs = tokenizer(sentence, return_tensors="pt")
    input_ids = inputs.input_ids.clone()  
    
    results = []
    
    for step in range(num_steps):
        with torch.no_grad():
            outputs = model(input_ids=input_ids)
            logits = outputs.logits
            
            probs = torch.softmax(logits[:, -1, :], dim=-1, dtype=torch.float32)
            
            top_probs, top_indices = torch.topk(probs, top_k, dim=-1)
            
            step_result = {
                '步骤': f'第{step+1}步',
                '当前句子': tokenizer.decode(input_ids[0], skip_special_tokens=True)
            }
            
            for i in range(top_k):
                token_id = top_indices[0][i].item()
                probability = top_probs[0][i].item() * 100
                token_text = tokenizer.decode([token_id])
                
                step_result[f'Top{i+1}_Token'] = token_text
                step_result[f'Top{i+1}_概率(%)'] = f'{probability:.2f}%'
            
            results.append(step_result)
            
            next_token_id = top_indices[0][0].unsqueeze(0).unsqueeze(0)
            input_ids = torch.cat([input_ids, next_token_id], dim=-1)
    
    df = pd.DataFrame(results)
    
    return df

def display_prediction_results(df):

    print("=" * 80)
    print("Token预测结果")
    print("=" * 80)
    
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 30)
    
    print(df.to_string(index=False))
    
    return df

def run_token_prediction(model, tokenizer, sentence, num_steps=10, top_k=5):

    try:
        results_df = predict_next_tokens(model, tokenizer, sentence, num_steps, top_k)
        formatted_df = display_prediction_results(results_df)
        return results_df
        
    except Exception as e:
        print(f"预测过程中出现错误: {e}")
        return None

df_result = run_token_prediction(CLLM, tokenizer, "hello,how are you")

### logits的理解

- 对任意句子(i have a )，进行tokenizer编码后(假设不包含special_token)，得到input_ids,长度为tokens_len
- 将input_ids输入到model中,即output = model(input_ids=input_ids),
- 得到output.logits,维度为(batch_size,tokens_len,vocab_size)
- 对于非最后一个token,即(batch_size,tokens_len[i],vocab_size),i!=-1,输出的含义:model更加偏向对该位置token的预测,
    - 例如(batch_size,tokens_len[0],vocab_size),是指model对第一个token(i)的预测概率的一个排序
- 而对于最后一个token,即(batch_size,tokens_len[-1],vocab_size),输出的含义:model更加偏向对下一个token的预测
    - 例如(batch_size,tokens_len[-1],vocab_size),更多的是对apple这个token的预测，而不是a这个token的预测

### 相关函数解释

- torch.argsort(),返回张量排序后的索引（indices）
    - 例如：lst = [3.1,-0.8,2.5],torch.argsort(lst),得到[1,2,0](升序顺序)
    - torch.argsort(lst,descending=True),得到[0,2,1](降序顺序)
- torch.argmax(),返回张量顺序后最大的索引
    - torch.argmax(lst),得到[0]
- model.generate()
    - max_length:prompt+generation的长度
    - max_new_tokens:generation的长度
    - 默认greed_search,如果beam_num不设置的话
- model.wte(input.input_ids)
    - 得到每一个token的编码后的向量
- model.wpe.weight
    - 得到位置编码向量

### 理解tokenizer.decode()

#### 如何理解tokenizer.decode()的时候，是tokenizer.decode(torch.argmax(lst)),是decode的索引,而不是logits
- 对于model(input_ids = input_ids).logits[:,-1,:],得到的是模型对于vocab再该位置输出token的概率
- 其中logits[:,-1,:]潜在的包含了顺序的关系，可以理解为logits[:,-1,:][0],表示的是对于词汇表token ids为0对应的token的概率
- 然后经过torch.argmax(lst)得到最大概率的索引,假设为100,那么logits[:,-1,:][100],是最大的概率对应的logits,那么根据上面可以得到logits[:,-1,:][100]表示的是对于词汇表第token ids为1000对应的token的概率
- 然后经过tokenizer.decode(torch.argmax(lst)),即tokenizer.decode([100]),就是相应的token

### outpout_hidden_state理解

llm = AutoModel.from_pretrained(model_name, trust_remote_code=True,output_hidden_states=True)
- output_hidden_states = True的理解
- 使用output_hidden_states = True后,llm(**input),会得到三个输出
    - last_hidden_state 最后一个hidden_states的输出
    - past_key_values KV键值缓存,维度：(batch_size, num_heads, sequence_length, embed_size_per_head)
    - hidden_states (output_hidden_states = True的结果) 所有隐层的输出
    - last_hidden_state == hidden_states[-1]